In [1]:
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
global gas
global info_data
#df =pd.read_csv("/datc/opschaler/smartmeter_data/P01S01W0373.csv",header=0,delimiter=";",parse_dates=['Timestamp', 'gasTimestamp'])

# Split the smartmeter and gasmeter data
#smart = df.iloc[:,:7]
#gas = df.iloc[:, 7:]

In [3]:

#These columns should be created in the end: column_names = ['House_ID','Amount_of_NaNs','Max_consecutive_NaNs', 'Interpolated','Days','File_size','Usable']#These  
#Create a df to store max consecutive NaNs

nan_dir = '//datc//opschaler//combined_gas_smart_weather_dfs//unprocessed//'
paths_h = glob.glob(nan_dir+'*_hour.csv')
paths_s = glob.glob(nan_dir+'*_10s.csv')

def extract_info(paths_list):
    info_data = pd.DataFrame(columns={'House_ID','Initial_Date','Final_Date','Days','Amount_of_NaNs','Max_Consecutive_NaNs'})
    for dwelling_name in tqdm(paths_list) :
        
        df=pd.read_csv(dwelling_name,delimiter='\t', parse_dates=['datetime'])
        df = df.set_index(['datetime'])
        df_nan=df.gasMeter.isnull()
        temp = pd.DataFrame()
        temp['temp']=df_nan.groupby((df_nan != df_nan.shift()).cumsum()).transform('size') * df_nan
        max_gap=temp['temp'].max()

        #Extract information about dates and number of NaNs
        initial_date_string= df.index[0].strftime('%Y-%m-%d %H:%M:%S')
        final_date_string= df.index[-1].strftime('%Y-%m-%d %H:%M:%S')
        days=df.index[-1]-df.index[0]
        NaN_count=df.gasMeter.isnull().sum()
        
        temp={'House_ID':dwelling_name[-20:],'Initial_Date':initial_date_string,'Final_Date':final_date_string,'Days':days,'Amount_of_NaNs':NaN_count,'Max_Consecutive_NaNs':max_gap}
        #Append all information to the global info_data dataframe
        info_data = info_data.append(temp, ignore_index=True)
        #info_data.loc[len(info_data.index)] = [dwelling_name[-20:], initial_date_string, final_date_string, days, NaN_count, max_gap]
    return (info_data)

In [10]:
testdf=pd.read_csv('/datc//opschaler//combined_gas_smart_weather_dfs//unprocessed//P01S01W0000_hour.csv',delimiter='\t',parse_dates=['datetime'])
testdf.gasMeter.isnull()

0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16    False
17    False
18     True
19     True
20     True
21     True
22     True
23     True
24    False
25    False
26    False
27    False
28    False
29    False
30    False
31    False
32    False
33    False
34    False
35    False
36    False
37    False
38    False
39    False
40    False
41    False
42    False
43    False
Name: gasMeter, dtype: bool

In [4]:
hourly_nans=extract_info(paths_h)
hourly_nans.to_csv('Hour_NaN_Gas_Table.csv', sep='\t', index=True,na_rep='NA')
#Ten_secs_nans=extract_info(paths_s)
#Ten_secs_nans.to_csv('Secs_NaN_Gas_Table.csv', sep='\t', index=True)

 89%|████████▉ | 50/56 [00:01<00:00, 26.47it/s]/opt/jupyterhub/anaconda/lib/python3.6/site-packages/pandas/core/computation/expressions.py:183: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  unsupported[op_str]))
100%|██████████| 56/56 [00:02<00:00, 27.31it/s]
